## Датасет собран из базы данных переписи 1994 года и содержит данные о доходах.
### Информация о данных:
* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, * Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
* salary: >50K,<=50K

## Проведите анализ данных при помощи Pandas выполнив поставленные задачи.
#### 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# загружаем датасет
data = pd.read_csv("./data/adult.data.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**1. Посчитайте, сколько мужчин и женщин (признак *sex*) представлено в этом датасете**

In [3]:
data.groupby('sex').agg({'age':'count'}).rename(columns={'age':'count'})

,count
sex,
Female,10771
Male,21790


**2. Каков средний возраст мужчин (признак *age*) по всему датасету?**

In [4]:
data.query(f'sex == "Male"')['age'].mean()

39.43354749885268

**3. Какова доля граждан Соединенных Штатов (признак *native-country*)?**

In [5]:
data['native-country'].value_counts(normalize=True, dropna=False)['United-States']

0.895857006848684

**4-5. Рассчитайте среднее значение и среднеквадратичное отклонение возраста тех, кто получает более 50K в год (признак *salary*) и тех, кто получает менее 50K в год**

In [6]:
data.groupby('salary').agg({'age':np.std}).rename(columns={'age':'age-std'})

,age-std
salary,
<=50K,14.020088
>50K,10.519028


**6. Правда ли, что люди, которые получают больше 50k, имеют минимум высшее образование? (признак *education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters* или *Doctorate*)**

In [7]:
[data.query('salary == ">50K"')['education'].value_counts().index] == ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

False

**7. Выведите статистику возраста для каждой расы (признак *race*) и каждого пола. Используйте *groupby* и *describe*. Найдите таким образом максимальный возраст мужчин расы *Asian-Pac-Islander*.**

In [8]:
data.groupby('race')['age'].describe().query('race == "Asian-Pac-Islander"')['max']

race
Asian-Pac-Islander    90.0
Name: max, dtype: float64

**8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак *marital-status*)? Женатыми считаем тех, у кого *marital-status* начинается с *Married* (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.**

In [9]:
data[['marital-status', 'salary']]\
    .apply(lambda x: pd.Series(["married",x[1]]) if 'Married-' in x[0] else pd.Series(["single",x[1]]),axis=1)\
    .rename(columns= {0:"status", 1:'salary'})\
    .value_counts(normalize=True)\
    .reset_index()\
    .query('salary == ">50K"')


,status,salary,0
2,married,>50K,0.206873
3,single,>50K,0.033936


**9. Какое максимальное число часов человек работает в неделю (признак *hours-per-week*)? Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?**

In [10]:
mhpw = data["hours-per-week"].max()
print(f'Максимальное число часов которое человек работает в неделю: {data["hours-per-week"].max()}')
print(f'Сколько людей работают такое количество часов -', data[data['hours-per-week'] == mhpw]['age'].count())
print('каков среди них процент зарабатывающих много -', data[data['hours-per-week'] == mhpw]['salary'].value_counts(normalize=True)['>50K'])

Максимальное число часов которое человек работает в неделю: 99
Сколько людей работают такое количество часов - 85
каков среди них процент зарабатывающих много - 0.29411764705882354


**10. Посчитайте среднее время работы (*hours-per-week*) зарабатывающих мало и много (*salary*) для каждой страны (*native-country*).**

In [11]:
data.groupby(['native-country','salary']).agg({'hours-per-week':'mean'})

hours-per-week
native-country salary                
?              <=50K        40.164760
               >50K         45.547945
Cambodia       <=50K        41.416667
               >50K         40.000000
Canada         <=50K        37.914634
...                               ...
United-States  >50K         45.505369
Vietnam        <=50K        37.193548
               >50K         39.200000
Yugoslavia     <=50K        41.600000
               >50K         49.500000

[82 rows x 1 columns]

**11.Сгруппируйте людей по возрастным группам *young*, *adult*, *retiree*, где:**
* *young* соответствует 16-35 лет
* *adult* - 35-70 лет
* *retiree* - 70-100 лет

**Проставьте название соответсвтуещей группы для каждого человека в новой колонке AgeGroup**

In [12]:
data['AgeGroup'] = data['age'].apply(lambda x: 'young' if 16<=x< 35 else 'adult' if 35<=x<70 else 'retiree' )

**12-13. Определите количество зарабатывающих >50K в каждой из возрастных групп (колонка AgeGroup), а также выведите название возрастной группы, в которой чаще зарабатывают больше 50К (>50K)**

In [13]:
data.query('salary == ">50K"').groupby('AgeGroup').agg({'age':'count'}).rename(columns={'age':'count'})

,count
AgeGroup,
adult,6240
retiree,113
young,1488


**14. Сгруппируйте людей по типу занятости (колонка occupation) и определите количество людей в каждой группе. После чего напишите функциюю фильтрации filter_func, которая будет возвращать только те группы, в которых средний возраст (колонка age) не больше 40 и в которых все работники отрабатывают более 5 часов в неделю (колонка hours-per-week)**

In [14]:
d = data.groupby('occupation').agg({'sex': 'count','age':'mean','hours-per-week':'min'}).rename(columns={'sex':'count'})
print(d['count'])
def filter_func(row):
    return row['age'] <= 40 and row['hours-per-week']>5

print(d[d.apply(filter_func,axis=1)])

occupation
?                    1843
Adm-clerical         3770
Armed-Forces            9
Craft-repair         4099
Exec-managerial      4066
Farming-fishing       994
Handlers-cleaners    1370
Machine-op-inspct    2002
Other-service        3295
Priv-house-serv       149
Prof-specialty       4140
Protective-serv       649
Sales                3650
Tech-support          928
Transport-moving     1597
Name: count, dtype: int64
              count        age  hours-per-week
occupation                                    
Armed-Forces      9  30.222222               8
